In [1]:
%%time
%run -i 'nlp_implementation.py' #imports packages and inits alltext
from pyspark import SparkContext, SparkConf
import spacy
from spacy.tokens import DocBin

conf = SparkConf().setAppName("pyspark-shell").setMaster('local[*]').set("spark.executor.memory", "10g").set("spark.driver.memory", "10g").set('spark.driver.maxResultSize', "10G")
sc = SparkContext(conf=conf)
nlp = spacy.load("en_core_web_lg")
with open("cucumber/all_doc.bin", "rb") as w: #loading in all bytestreams of data
    new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
docs1 = list(new_docs1.get_docs(nlp.vocab))
with open("cucumber/first_doc.bin", "rb") as w: #loading in
    new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
docs1 += list(new_docs1.get_docs(nlp.vocab))
print(len(docs1))

145
CPU times: user 33.1 s, sys: 2.83 s, total: 36 s
Wall time: 39.5 s


In [5]:
%%time
import wikipedia
# from googlesearch import search
import requests
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
import numpy as np

import os
import multiprocessing as mp
import subprocess
import concurrent.futures
from difflib import SequenceMatcher
import csv

books_df = pd.read_csv('books_clean.csv')
def is_book1(name, df=books_df): #worker
#     print("trying", name)
    db, wiki, = False, False  
    if name in df.title.values:
        db = True
    similar = ["book", "volume", "novel", "work", "publication", "title", "treatise"]
    try:
        summ = wikipedia.summary(name, sentences=3)
        if any([x in summ.lower() for x in similar]):
            wiki =True
    except:
        pass

    if db or wiki:
        print(name)
        return(name, True)
#     links = search(name)
#     websites_matched = 0
#     for l in links:
#         if bool(re.search("amazon.*dp", l)):
#             websites_matched+=1
#         if bool(re.search("books\.google.*"+name.replace(" ", "_").lower(), l.lower())):
#             websites_matched+=1
#         if bool(re.search("goodreads*"+name.replace(" ", "_").lower(), l.lower())):
#             websites_matched+=1
#         if bool(re.search("barnesandnoble*"+name.replace(" ", "_").lower(), l.lower())):
#             websites_matched+=1
#         if bool(re.search("penguinrandomhouse*"+name.replace(" ", "_").lower(), l.lower())):
#             websites_matched+=1

#     if websites_matched>2 or sum([bool(re.search("book", l.lower())) for l in links])>4:
#         return (name, True) 
    
    return (name, False)

        
def keep_stuff1(doc):
    books, people, places = [], [], []
    ents = doc.user_data["entis"]
    potential = ["WORK_OF_ART"]
    
    for ent, label in ents:
        if label=="PERSON": #save people entities
            people.append(ent)
        elif label=="LOC" or label=="GPE": #save location entites
            places.append(ent)
    #parallel processing to verify books
    with concurrent.futures.ThreadPoolExecutor(max_workers = 20) as executor:
        result = [executor.submit(is_book1, e[0]) for e in ents if e[1] in potential]
    for future in concurrent.futures.as_completed(result):
        print(future.result())
        if future.result()[1]==True:
            books.append(future.result()[0])
    doc.user_data["places"] = places
    doc.user_data["people"] = people
    doc.user_data["books"] = books
#     doc.user_data["people"] = [person[0] for person in ents if person[1]=="PERSON"]
#     doc.user_data["place"] = [place[0] for place in ents if place[1]=="LOC" or place[1]=="GPE"]
#     doc.user_data["book"] = [book[0] for book in ents if book[1] in potential]
    
    
    return doc
docs1 = [keep_stuff1(doc) for doc in docs1] #extract and save all entities by running above method

Bible
Deep Work
Bible
('Bible', True)
('Consciousness, Religion and Gurus Pitfalls of Psychedelic Medicine', False)
('Burning Man', False)
('Koonce', False)
('Deep Work', True)
('the Nobel Prize', False)
('Lack', False)
('Eye-Opening', False)
('Nobel Prizes', False)
('The Varieties of Psychedelic Experience', False)
('Bible', True)
PhD
Pollyanna
PhDPh.D.

('a Pulitzer Prize', False)
('How Emotions Are Made', False)
('The Human Brain', False)
('PhD', True)
('PhD', True)
('Pollyanna', True)
('Ph.D.', True)
('the The Storied Life of A.J. Fear Key', False)
('Naming the Mind by', False)
Ph.D.
Denial of Death
The Year of Magical Thinking
The Verge
('The Year of Magical Thinking', True)
('The Verge', True)
('Ph.D.', True)
('Denial of Death', True)
('The Most Human', False)
('Loebner Prize', False)
('Nintendo', False)
Bible
BibleBible
Bible
Bible

('Please Allow Me', False)
('Bible', True)
('The Town', False)
('Magic Spoon', False)
('Bible', True)
('Bible', True)
('Bible', True)
('Bible', True

In [12]:

def att_to_csv(docs, atts):
    all_attributes= sorted(set([item for sublist in docs for item in sublist.user_data[atts]]))
    i = 0
    
#     similar_words implement dictionary to store similar words that were deleted
    while i<len(all_attributes)-1: #remove similar or subwords
        str1, str2 = all_attributes[i], all_attributes[i+1]
        if SequenceMatcher(a=str1,b=str2).ratio()>.8:
            all_attributes.remove(str1) if len(str1)>len(str2) else all_attributes.remove(str2)
        if str1 in str2:
            all_attributes.remove(str1)
        elif str2 in str1:
            all_attributes.remove(str2)
        i+=1
    
    all_attributes_dict = dict(zip(all_attributes, np.arange(len(all_attributes)))) #create dict for all entities
    
    
    guests = [doc.user_data["guest"] for doc in docs]
    guests_dict = dict(zip(guests, np.arange(len(all_attributes), len(all_attributes)+len(guests))))#create dict for all main
    
    #now create dict for all edges by going from every doc's mentions of attribute, and adding edge from guest to attribute
    
    i = len(guests) + len(all_attributes) #make keys for final edges
    
    edges = []
    for doc in docs:
        current_name = guests_dict.get(doc.user_data["guest"]) #get graph id for each guest in doc
        for mention in doc.user_data[atts]:
            if all_attributes_dict.get(mention):
                edges.append((current_name, all_attributes_dict.get(mention))) #from speaker to mention
    edges_dict=dict(zip(edges, np.arange(i, i+len(edges))))
    
    
    
    with open(atts+"Nodes.csv", "w+") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["id", "name", "group"])
        for key, value in all_attributes_dict.items(): #key is attriubte name, value is id
            writer.writerow([value, key, 2])
        for key, value in guests_dict.items():
            writer.writerow([value, key, 1]) # group 2 is entities, group 1 is speakers
    with open(atts+"Edges.csv", "w+") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["id", "source", "target", "value"])
        for key, value in edges_dict.items():
            writer.writerow([value, key[0], key[1], 1])
att_to_csv(docs1, "people")

In [9]:
# att_to_csv(docs1, "places")
# att_to_csv(docs1, "books")
from collections import Counter

all_attributes= Counter([item for sublist in docs1 for item in sublist.user_data["books"]])
for item in all_attributes.items():
    print(item)

('Bible', 37)
('Deep Work', 2)
('PhD', 21)
('Pollyanna', 1)
('Ph.D.', 37)
('The Year of Magical Thinking', 1)
('The Verge', 1)
('Denial of Death', 5)
('Write Me a Poem', 1)
('Hitchhiker’s Guide', 1)
('The Worldly Philosophers', 1)
('The Piano Man Nice', 1)
('Sing Me', 1)
('the Measure of Intelligence', 1)
('E.T. and Close Encounters', 1)
('The Bottom Line', 1)
('Strange Land and YouTube', 1)
('Top Gun', 3)
('Star Trek', 19)
('Tom Cruise', 1)
('Project Blue Book', 2)
('A New Kind of Science', 2)
('A Project to Find the Fundamental Theory of Physics', 1)
('The Innovators', 1)
('Ender’s Game', 2)
('The Republic', 1)
('Dune', 3)
('Uptown Girl', 2)
('the The Dictionary of Curious and Interesting Numbers', 1)
('The War of Art', 2)
('Deep Questions', 1)
('Cuddle Parties', 1)
('Your Welcome', 1)
('A Course in Miracles', 1)
('Apprentice', 1)
('Superman', 9)
('The Crowd', 1)
('Birth of a Nation', 1)
('Trump', 3)
('The Origin of Species', 1)
('a MacBook Air', 1)
('The Metamorphosis of Prime Intel

In [17]:
import requests
from bs4 import BeautifulSoup
import re
search = "spongebob"
results = 10 # valid options 10, 20, 30, 40, 50, and 100
page = requests.get("https://www.duckduckgo.com/?q="+search+"&num="+str(results))
# print(page.text)
soup = BeautifulSoup(page.content, "html.parser")
print(soup.prettify())
links = soup.findAll("a")
for link in links :
    link_href = link.get('href')
    print(link_href)
#     if "url?q=" in link_href and not "webcache" in link_href:
#         print(link.get('href').split("?q=")[1].split("&sa=U")[0])

<!DOCTYPE html>
<!--[if IEMobile 7 ]> <html lang="en_US" class="no-js iem7"> <![endif]-->
<!--[if lt IE 7]> <html class="ie6 lt-ie10 lt-ie9 lt-ie8 lt-ie7 no-js" lang="en_US"> <![endif]-->
<!--[if IE 7]>    <html class="ie7 lt-ie10 lt-ie9 lt-ie8 no-js" lang="en_US"> <![endif]-->
<!--[if IE 8]>    <html class="ie8 lt-ie10 lt-ie9 no-js" lang="en_US"> <![endif]-->
<!--[if IE 9]>    <html class="ie9 lt-ie10 no-js" lang="en_US"> <![endif]-->
<!--[if (gte IE 9)|(gt IEMobile 7)|!(IEMobile)|!(IE)]><!-->
<html class="no-js" lang="en_US">
 <!--<![endif]-->
 <head>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8;charset=utf-8" http-equiv="content-type"/>
  <meta content="width=device-width, initial-scale=1, user-scalable=1" name="viewport">
   <meta content="true" name="HandheldFriendly"/>
   <link href="https://duckduckgo.com/50x.html" rel="canonical"/>
   <link href="/s1943.css" rel="stylesheet" type="text/css"/>
   <link href="/o1943.css" rel="